<a href="https://colab.research.google.com/github/IFuentesSR/RSE_drought/blob/main/exporting_gridded_toCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab for saving SPEI-SRI (or others) tiles

This colab was created to export tiles of SPEI-SRI collections as multiband tiles given the limitations in computing global lagged responses between both indices. These tiles were downloaded to google cloud storage and then processed using python scripts in a High Performance Computer to parallelise the analysis.

In [1]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=HFn2_fZnz9UDaOCgxQi8Djq8UJ1xToSkjbZkdgraJqQ&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6Gx47WhlE2iJ3qYX9vMoFpybv55bmpGM7p5rcCSYMoLm5P1xn77Aw

Successfully saved authorization token.


In [2]:
import folium
import time


print(folium.__version__)

0.8.3


### Defining functions for tiling the data

In [3]:
def generateGrid(xmin, ymin, xmax, ymax, dx, dy, marginx, marginy):
  xx = ee.List.sequence(xmin, ee.Number(xmax).subtract(ee.Number(dx).multiply(0.9)), dx)
  yy = ee.List.sequence(ymin, ee.Number(ymax).subtract(ee.Number(dy).multiply(0.9)), dy)
  
  def innerX(x):
    def innerY(y):
      x1 = ee.Number(x).subtract(marginx)
      x2 = ee.Number(x).add(ee.Number(dx)).add(marginx)
      y1 = ee.Number(y).subtract(marginy)
      y2 = ee.Number(y).add(ee.Number(dy)).add(marginy)
      coords = ee.List([x1, y1, x2, y2])
      rect = ee.Algorithms.GeometryConstructors.Rectangle(coords, 'EPSG:4326', False)
      return ee.Feature(rect)
    return yy.map(innerY)
  cells = xx.map(innerX).flatten()

  return ee.FeatureCollection(cells)


def parse_id(fea):
  return fea.set('id', ee.Number.parse(fea.id()))

### Grid definition

In [4]:
land = ee.FeatureCollection('users/ignisfausto/Land_border')
grid = generateGrid(-180, -56, 180, 60, 10, 10, 0, 0)
grid = grid.map(parse_id)
grid = grid.filterBounds(land)

In [7]:
mapid = ee.FeatureCollection(grid).getMapId({'opacity':0.1})
map = folium.Map(location=[-30.63, 149.91], zoom_start=2)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='grid',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

### Loading Multiband (time series) images

In [9]:
image = ee.Image("users/ignisfausto/SPEI_3month")
image2 = ee.Image("users/ignisfausto/SSI3_ERA5")

bands1 = image.bandNames()
bands2 = image2.bandNames()

### Trnsforming multiband images to Image Collections

In [12]:
def set_date(band):
    img = image.select([band])
    date = ee.Date(ee.String(band).slice(4).replace('_', '-').replace('_', '-'))
    return img.rename('SPEI').setMulti({'system:time_start':date.millis(), 'date':date.format('YYYY_MM_dd')})

SPEI = bands1.map(set_date)
SPEI = ee.ImageCollection.fromImages(SPEI).sort('system:time_start', True)

def set_date2(band):
    img = image2.select([band])
    date = ee.Date(ee.String(band).slice(4).replace('_', '-').replace('_', '-'))
    return img.rename('SRI').setMulti({'system:time_start':date.millis(), 'date':date.format('YYYY_MM_dd')})

SRI = bands2.map(set_date2)
SRI = ee.ImageCollection.fromImages(SRI).sort('system:time_start', True)



### Merging Image Collections and creating single multiband image

In [15]:
innerJoined = ee.Join.inner().apply(primary=SPEI,
                                    secondary=SRI,
                                    condition=ee.Filter.equals(leftField='system:time_start',
                                                               rightField='system:time_start'))
def mergeImageBands1(joinResult):
    spei = ee.Image(joinResult.get('primary'))
    return spei


def mergeImageBands2(joinResult):
    spei = ee.Image(joinResult.get('primary'))
    svi = ee.Image(joinResult.get('secondary')).updateMask(spei.gt(-999))
    return svi


merged1 = ee.ImageCollection(innerJoined.map(mergeImageBands1))
merged2 = ee.ImageCollection(innerJoined.map(mergeImageBands2))

dates = merged1.aggregate_array('date')

def set_band_names1(date):
    return ee.String('SPEI').cat(date)


def set_band_names2(date):
    return ee.String('SRI').cat(date)

new_bands1 = dates.map(set_band_names1)
new_bands2 = dates.map(set_band_names2)

toExp = merged1.toBands().rename(new_bands1).addBands(merged2.toBands().rename(new_bands2))
print(toExp.bandNames().getInfo())



['SPEI1982_02_01', 'SPEI1982_03_01', 'SPEI1982_04_01', 'SPEI1982_05_01', 'SPEI1982_06_01', 'SPEI1982_07_01', 'SPEI1982_08_01', 'SPEI1982_09_01', 'SPEI1982_10_01', 'SPEI1982_11_01', 'SPEI1982_12_01', 'SPEI1983_01_01', 'SPEI1983_02_01', 'SPEI1983_03_01', 'SPEI1983_04_01', 'SPEI1983_05_01', 'SPEI1983_06_01', 'SPEI1983_07_01', 'SPEI1983_08_01', 'SPEI1983_09_01', 'SPEI1983_10_01', 'SPEI1983_11_01', 'SPEI1983_12_01', 'SPEI1984_01_01', 'SPEI1984_02_01', 'SPEI1984_03_01', 'SPEI1984_04_01', 'SPEI1984_05_01', 'SPEI1984_06_01', 'SPEI1984_07_01', 'SPEI1984_08_01', 'SPEI1984_09_01', 'SPEI1984_10_01', 'SPEI1984_11_01', 'SPEI1984_12_01', 'SPEI1985_01_01', 'SPEI1985_02_01', 'SPEI1985_03_01', 'SPEI1985_04_01', 'SPEI1985_05_01', 'SPEI1985_06_01', 'SPEI1985_07_01', 'SPEI1985_08_01', 'SPEI1985_09_01', 'SPEI1985_10_01', 'SPEI1985_11_01', 'SPEI1985_12_01', 'SPEI1986_01_01', 'SPEI1986_02_01', 'SPEI1986_03_01', 'SPEI1986_04_01', 'SPEI1986_05_01', 'SPEI1986_06_01', 'SPEI1986_07_01', 'SPEI1986_08_01', 'SPEI1986

In [18]:
vectors = grid.getInfo()

### Exporting tiles to google cloud storage

In [ ]:
for i, n in enumerate(vectors['features'][:]):
    ee.batch.Export.image.toCloudStorage(image=toExp,
                                  region=ee.Geometry(n['geometry']),
                                  description='spei_sri_{}'.format(n['id']),
                                  scale=25000,
                                  bucket='files_drought',
                                  maxPixels=1e13).start()
    time.sleep(40)
